## Integrals Over Infinite Ranges

* Change of variables
    - For the range $(0,\infty)$, use $z=x/(1+x)$ or $z=x/(c+x)$ (for any constant value $c$) substitution.
    - For the range $(a,\infty)$, use $y=x-a$ and then $z=y/(c+y)$ substitution.
    - For the range $(-\infty,a)$, use $z\rightarrow -z$ in the above substitution.
    - For the range $(-\infty,\infty)$, divide the range into $(-\infty,0)$ and $(0,\infty)$ or $(-\infty,a)$ and $(a,\infty)$. Alternatively, use $x=\tan z$.

## Integrate $\int_0^\infty e^{-t^2}$

In [2]:
import numpy as np

# Function to calculate sample points an weights for Gaussian quadrature
def gaussxw(N): # N is the number of sample points and order of Gaussian approximation to the integral

    # Initial approximation to roots of the Legendre polynomial
    a = np.linspace(3,4*N-1,N)/(4*N+2)
    x = np.cos(np.pi*a+1/(8*N*N*np.tan(a)))

    # Finding roots (or nodes) using Newton's method (will be taught soon)
    epsilon = 1e-15
    delta = 1.0
    while delta>epsilon:
        p0 = np.ones(N,float)
        p1 = np.copy(x)
        for k in range(1,N):
            p0,p1 = p1,((2*k+1)*x*p1-k*p0)/(k+1)
        dp = (N+1)*(p0-x*p1)/(1-x*x)
        dx = p1/dp
        x -= dx
        delta = max(abs(dx))

    # Calculating the weights using standard interval for Legendre polynomial
    w = 2*(N+1)*(N+1)/(N*N*(1-x*x)*dp*dp)

    return x,w

In [ ]:
true_val = np.sqrt(np.pi)/2

def f(z):
    t = z/(1-z)
    return 

N = 50
a = 0.0
b = 

x, w = gaussxw(N)
xp = 0.5*(b-a)*x + 0.5*(b+a)
wp = 0.5*(b-a)*w

s = 0.0
for k in range(N):
    s += 
    
print(s,true_val)

## Multiple Integrals

The easiest thing to do is to compute integrations over different quantities one after another. Applying the Gaussian quadrature method, for example, gives the Gauss-Legendre product formula.
$$ I = \int_0^1\int_0^1 f(x,y) \ dx\ dy \simeq \sum_{i=1}^N\sum_{i=1}^N w_iw_jf(x_i.y_j) $$

### Gravitational pull of a uniform sheet

The gravitational force due to a square sheet at some distance $z$ from it is given by
$$ F_z = G\sigma z\int\int_{-L/2}^{L/2}\frac{dx\ dy}{(x^2+y^2+z^2)^{3/2}} $$
where G is the gravitational constant and $\sigma$ is the mass per unit area.  
The sheet weighs $10^4$ kg and has a side measuring 10 m.

In [ ]:
import matplotlib.pyplot as plt

G     = 6.67e-11   # gravitational constant in SI unit
sigma = 100        # mass per unit area of the sheet

def r3inv(x,y,z):
    return 

nz = 100
z  = np.linspace(0,100,nz) # fixed z-axis
Fz = np.zeros(nz)         # creating an empty list to store force at some z

N = 100
a = 
b = 

x, w = gaussxw(N)
xp = yp = 0.5*(b-a)*x + 0.5*(b+a)
wp = 0.5*(b-a)*w

for k in range(nz):    # calculating force Fz for each z
    s = 0.0
    for j in range(N):
        for i in range(N):
            s += 
    Fz[k] = 

# plotting force vs z
plt.plot(z,Fz,'k')
plt.yscale('log')
plt.xlabel(r'$z$')
plt.ylabel(r'$F_z$')
plt.show()

## Derivatives

* Developed later, since most derivatives can be calculated analytically
* Unavoidable errors

* Forward difference:
$$ f^\prime(x) \simeq \frac{f(x+h)-f(x)}{h} $$
* Backward difference:
$$ f^\prime(x) \simeq \frac{f(x)-f(x-h)}{h} $$
* Central difference:
$$ f^\prime(x) \simeq \frac{f(x+h/2)-f(x-h/2)}{h} $$

### Error analysis

Prone to both roubding and approximation errors.  
* Forward/backward difference $\epsilon \sim \frac{1}{2}hf^{\prime\prime}(x) $
* Central difference $\epsilon \sim \frac{1}{24}h^2f^{\prime\prime\prime}(x) $

For **Sampled Function**, central difference is only better if:
$$ h < \left|\frac{f^{\prime\prime}(x)}{f^{\prime\prime\prime}(x)} \right| $$

In [ ]:
import matplotlib.pyplot as plt

def func(x):
    return 1+0.5*np.tanh(2*x)

h = 0.1
a, b = -2, 2
x = np.arange(a,b,h)
plt.plot(x,func(x))

In [ ]:
y = func(x)
dydx = []

# forward difference
for i in range():
    dydx.append()
plt.plot(x[:-1],dydx)

dydx = []
# backward difference
for i in range():
    dydx.append()
plt.plot(x[1:],dydx)

dydx = []
# central difference
for i in range():
    dydx.append()
plt.plot(x[1:-1],dydx)

In [ ]:
def func_deriv(x):
    return 1 - np.tanh(2*x)**2

plt.plot(x,func_deriv(x))

# Try it yourself

### Total 4 marks

Try h = {0.01, 0.001, 0.0001} for all the cases above. Repeat the calculation with forward and central difference. Calculate the error in each case, using the analytical expression and plot them against h. Does this investigation prove the condition for sampled functions:
$$ h < \left|\frac{f^{\prime\prime}(x)}{f^{\prime\prime\prime}(x)} \right| $$